In [15]:
from torchcnnbuilder.models import ForecasterBase
from torchcnnbuilder.preprocess.time_series import multi_output_tensor

import os
from datetime import datetime
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [16]:
device = 'mps'
print(f'Calculating on device: {device}')

Calculating on device: mps


In [17]:
batch_size = 10
epochs = 500
learning_rate = 1e-3

data_freq = 7

In [18]:
x_virg = []
temp_ar = []
for file in os.listdir('/Users/kdduha/projects/osisaf/train'):
    date = datetime.strptime(file, 'osi_iceconc_%Y%m%d.npy')
    if date.year < 2013:
        array = np.load(f'/Users/kdduha/projects/osisaf/train/{file}')
        temp_ar.append(array)
        if len(temp_ar) == data_freq:
            temp_ar = np.array(temp_ar)
            temp_ar = temp_ar[-1]
            x_virg.append(temp_ar)
            temp_ar = []

In [19]:
for file in os.listdir('/Users/kdduha/projects/osisaf/test'):
    date = datetime.strptime(file, 'osi_iceconc_%Y%m%d.npy')
    if date.year < 2013:
        array = np.load(f'../../matrices/osisaf/test/{file}')
        temp_ar.append(array)
        if len(temp_ar) == data_freq:
            temp_ar = np.array(temp_ar)
            temp_ar = temp_ar[-1]
            x_virg.append(temp_ar)
            temp_ar = []

In [20]:
x_virg = np.array(x_virg)
x_virg.shape

(1200, 125, 125)

In [21]:
pre_history_size = 104
forecast_size = 52

In [22]:
dataset = multi_output_tensor(data=x_virg,
                             forecast_len=forecast_size,
                             pre_history_len=pre_history_size)

In [23]:
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [24]:
model = ForecasterBase(input_size=[125, 125],
                       n_layers=5,
                       in_channels=pre_history_size,
                       out_channels=forecast_size,
                       normalization='batchnorm').to(device)

In [25]:
model

ForecasterBase(
  (convolve): Sequential(
    (conv 1): Sequential(
      (0): Conv2d(104, 116, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(116, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (conv 2): Sequential(
      (0): Conv2d(116, 128, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (conv 3): Sequential(
      (0): Conv2d(128, 140, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(140, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (conv 4): Sequential(
      (0): Conv2d(140, 152, kernel_size=(3, 3), stride=(1, 1))
      (1): BatchNorm2d(152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (conv 5): Sequential(
      (0): Conv2d(152, 164, kernel_size=(3, 3), stride=(1, 1))
 

In [26]:
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.L1Loss()

for epoch in range(epochs):
    loss = 0
    for train_features, test_features in dataloader:
        train_features = train_features.to(device)
        test_features = test_features.to(device)
        optimizer.zero_grad()
        outputs = model(train_features)
    
        l1_loss = criterion(outputs, test_features)
        train_loss = l1_loss
        
        train_loss.backward()
        optimizer.step()
        
        loss += train_loss.item()

    loss = loss / len(dataloader)

    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))

epoch : 1/500, recon loss = 0.32088170
epoch : 2/500, recon loss = 0.23731762
epoch : 3/500, recon loss = 0.20967886
epoch : 4/500, recon loss = 0.18363650
epoch : 5/500, recon loss = 0.18003846
epoch : 6/500, recon loss = 0.19205401
epoch : 7/500, recon loss = 0.16664704
epoch : 8/500, recon loss = 0.16110674
epoch : 9/500, recon loss = 0.16497280
epoch : 10/500, recon loss = 0.15074425
epoch : 11/500, recon loss = 0.14246412
epoch : 12/500, recon loss = 0.13899708
epoch : 13/500, recon loss = 0.13586665
epoch : 14/500, recon loss = 0.13249330
epoch : 15/500, recon loss = 0.12921784
epoch : 16/500, recon loss = 0.12694446
epoch : 17/500, recon loss = 0.12611524
epoch : 18/500, recon loss = 0.12530524
epoch : 19/500, recon loss = 0.12392265
epoch : 20/500, recon loss = 0.12235307
epoch : 21/500, recon loss = 0.12067000
epoch : 22/500, recon loss = 0.12026559
epoch : 23/500, recon loss = 0.12010195
epoch : 24/500, recon loss = 0.12002629
epoch : 25/500, recon loss = 0.11994468
epoch : 2

KeyboardInterrupt: 

Вроде forward и backward работают адекватно, проверял модель с нормализацией (последний запуск) + без нормализации как в статье (первичный запуск), довел до эпох 40 и бросил 